# Montana Board of Oil and Gas Conservation

The purpose of this notebook is to outline the process for collecting oil and gas data from the state of Montana, and data from frac focus. We will then write a subset of this data with horizontal well completions to .csv files that we will use in other notebooks for EDA and modelling. 

In [1]:
# importing libraries
import pandas as pd
import seaborn as sns
from pull_data import pull_ff_data, pull_prod_data, pull_well_data

In [ ]:
# pull data
lease_prod_df, well_prod_df = pull_prod_data
well_data_df = pull_well_data



# View Data
Let's start by taking a look at these 3 files pulled from the .zip file. There are three flat files. `lease_prod_df` contains production reported on a lease basis, `well_prod_df` contains production on a per well monthly basis, and `well_data_df` contains the well information (Producting status, field, horizontal, vertical drill profile, etc.)

In [72]:
lease_prod_df.head()

,Lease_Unit,Rpt_Date,Dt_Receive,Del_Rpt,Amnd_Rpt,OpNo,CoName,StartIvn_OilCd,Oil_Prod,Gas_Prod,...,WtrInj,WtrTo_Pit,Other_Oil,Other_Gas,Other_Wtr,Dt_Amend,Lease_Update,No_ProdWells,No_SIWells,Dt_Mod
0,2,01/31/2001,03/15/2001,False,False,5385,EnCana Oil & Gas (USA) Inc.,0.0,0.0,353.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,1.0,0.0,05/18/2001
1,3,01/31/2001,03/15/2001,False,False,5385,EnCana Oil & Gas (USA) Inc.,0.0,0.0,69.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,1.0,0.0,05/18/2001
2,4,01/31/2001,03/05/2001,False,False,6681,Samedan Oil Corporation,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,1.0,03/06/2001
3,5,01/31/2001,03/05/2001,False,False,6681,Samedan Oil Corporation,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,1.0,03/06/2001
4,6,01/31/2001,03/05/2001,False,False,6681,Samedan Oil Corporation,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,1.0,03/06/2001


In [73]:
lease_prod_df.describe()

,Lease_Unit,OpNo,StartIvn_OilCd,Oil_Prod,Gas_Prod,Wtr_Prod,Oil_Sold,Gas_Sold,OilSpill,WtrSpill,...,UseGas,OilInj,GasInj,WtrInj,WtrTo_Pit,Other_Oil,Other_Gas,Other_Wtr,No_ProdWells,No_SIWells
count,1.298191e+06,1.298191e+06,1.283575e+06,1.298181e+06,1.297562e+06,1.297373e+06,1.297370e+06,1.298186e+06,1.297238e+06,78134.000000,...,1.298188e+06,1.297233e+06,1.298187e+06,1.279484e+06,1.279527e+06,1.286605e+06,1.298187e+06,1.279363e+06,1.298170e+06,1.288983e+06
mean,1.370504e+04,1.717545e+03,9.998504e+01,4.047499e+02,1.476859e+03,2.764494e+03,4.047213e+02,1.132646e+03,3.033368e-03,0.020900,...,4.778551e+01,3.475860e-03,2.352886e+02,1.616128e+03,3.201118e+02,1.094641e-01,3.267775e+00,2.214530e+02,1.815966e+00,7.251414e-01
std,3.718679e+04,2.344972e+03,2.078206e+02,3.377169e+03,3.272934e+04,3.604075e+04,3.378501e+03,9.589917e+03,6.614214e-01,5.485964,...,4.437782e+02,1.120417e+00,3.114522e+04,3.160498e+04,9.859811e+03,1.468871e+01,1.465689e+02,8.156987e+03,1.024098e+01,4.598063e+00
min,2.000000e+00,4.000000e+00,-8.150000e+02,-4.020000e+02,0.000000e+00,-5.150000e+02,-2.300000e+02,0.000000e+00,0.000000e+00,0.000000,...,-8.190000e+02,-4.800000e+01,0.000000e+00,-9.300000e+01,-5.000000e+01,-8.580000e+03,-5.540000e+02,-7.055000e+04,0.000000e+00,0.000000e+00
25%,2.666000e+03,3.210000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
50%,5.251000e+03,5.360000e+02,0.000000e+00,0.000000e+00,1.380000e+02,0.000000e+00,0.000000e+00,6.900000e+01,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
75%,7.387000e+03,1.720000e+03,1.460000e+02,1.170000e+02,7.100000e+02,1.250000e+02,1.220000e+02,5.980000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,9.900150e+05,1.230600e+04,8.493000e+03,2.318970e+05,5.934068e+06,6.684198e+06,2.319860e+05,6.788670e+05,2.480000e+02,1530.000000,...,7.351400e+04,9.890000e+02,5.934068e+06,2.096867e+06,6.684198e+06,3.000000e+03,2.473100e+04,8.252006e+06,1.154000e+03,1.200000e+03


In [74]:
well_prod_df.head()

,rpt_date,API_WELLNO,ST_FMTN_CD,Name_,Lease_Unit,OPNO,CoName,BBLS_OIL_COND,MCF_GAS,BBLS_WTR,DAYS_PROD,AMND_RPT,STATUS,dt_mod
0,12/30/1899,25035061990000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
1,12/30/1899,25035062030000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
2,12/30/1899,25035062220000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
3,12/30/1899,25035062380000,CB,Cut Bank,2593.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/13/2020
4,12/30/1899,25035062860000,CB,Cut Bank,2203.0,1770.0,Cut Bank Gas Company,0.0,0.0,0,0.0,False,NaN,04/10/2020


In [75]:
well_prod_df.describe()

,API_WELLNO,Lease_Unit,OPNO,BBLS_OIL_COND,MCF_GAS,BBLS_WTR,DAYS_PROD
count,5.028322e+06,5.028292e+06,4.181218e+06,5.028320e+06,5.028321e+06,5.028322e+06,5.028249e+06
mean,2.506244e+13,8.618911e+03,1.789600e+03,1.612155e+02,5.443820e+02,1.058946e+03,1.940332e+01
std,3.397980e+10,3.509633e+04,2.436512e+03,6.258063e+02,2.798577e+03,6.467663e+03,1.495651e+01
min,2.500305e+13,-9.990000e+02,4.000000e+00,-4.020000e+02,-6.050000e+02,-4.120000e+03,-3.400000e+01
25%,2.503506e+13,1.583000e+03,3.210000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.507122e+13,2.785000e+03,5.310000e+02,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01
75%,2.509960e+13,4.829000e+03,2.323000e+03,2.800000e+01,4.490000e+02,6.100000e+01,3.100000e+01
max,2.511192e+13,9.900150e+05,1.230600e+04,4.519700e+04,3.429430e+05,2.944924e+06,3.130000e+03


In [76]:
well_data_df.head()

,API_WellNo,OpNo,CoName,Well_Nm,Well_Typ,Type,Wl_Status,Status,Wh_Sec,Wh_Twpn,...,Wh_EW,Slant,Reg_Field_No,Reg_Field,Stat_Field_No,Stat_Field,Dt_APD,Dt_Cmp,Elev_KB,DTD
0,25101100140000,7120,"Somont Oil Company, Inc.",Remington-Warner 7,OIL,Oil,PR,Producing,32,35,...,W,V,4820,Kevin-Sunburst,4820,Kevin-Sunburst,NaN,1927-07-05 00:00:00,NaN,1500.0
1,25025225200000,664,"Denbury Onshore, LLC",Little Beaver East 23-22H,OIL,Oil,SI,Shut In,22,5,...,W,H,5420,"Little Beaver, East",5420,"Little Beaver, East",2005-04-12 00:00:00,2006-01-27 00:00:00,2989.0,11250.0
2,25073210450000,5130,Mont Mil Operating Company,TMCBSU 10-10,EOR,"Injection, EOR",AX,P&A - Approved,10,31,...,W,V,2400,Cut Bank,2400,Cut Bank,1968-11-25 00:00:00,1969-07-15 00:00:00,NaN,3422.0
3,25065055920000,645,Kelly Oil and Gas LLC,Smith M #3,OIL,Oil,SI,Shut In,12,10,...,E,V,4700,Keg Coulee,4700,Keg Coulee,1966-10-20 00:00:00,1966-12-17 00:00:00,NaN,4855.0
4,25101226430000,4070,Kipling Energy Incorporated,Allen 4,OIL,Oil,AX,P&A - Approved,11,34,...,E,V,4820,Kevin-Sunburst,4820,Kevin-Sunburst,1984-07-20 00:00:00,1984-08-16 00:00:00,NaN,1588.0


In [77]:
well_data_df.describe()

,API_WellNo,OpNo,Wh_Sec,Wh_Twpn,Wh_RngN,Wh_FtNS,Wh_FtEW,Reg_Field_No,Stat_Field_No,Elev_KB,DTD
count,1.928100e+04,19281.000000,19281.000000,19281.000000,19281.000000,19257.000000,19257.000000,19281.000000,19281.000000,9045.000000,19229.000000
mean,2.505986e+13,1659.379648,18.664281,25.581920,27.848400,1290.213792,1334.870541,5244.297806,4405.151289,2996.814594,4281.091892
std,3.456461e+10,2364.241800,10.166079,11.176755,22.138558,711.827299,685.701671,3383.384804,2756.568076,635.346835,4731.850855
min,2.500305e+13,4.000000,1.000000,1.000000,1.000000,0.000000,0.000000,5.000000,5.000000,1890.000000,89.000000
25%,2.502523e+13,369.000000,10.000000,13.000000,5.000000,660.000000,691.000000,2400.000000,2400.000000,2456.000000,1505.000000
50%,2.507122e+13,676.000000,19.000000,31.000000,29.000000,1229.000000,1315.000000,4820.000000,4120.000000,2963.000000,2000.000000
75%,2.509121e+13,891.000000,27.000000,35.000000,54.000000,1972.000000,1980.000000,8100.000000,6680.000000,3478.000000,4650.000000
max,2.511192e+13,12317.000000,36.000000,37.000000,62.000000,7090.000000,4937.000000,11111.000000,11111.000000,6252.000000,24821.000000


# Clean

Remove wells and leases that have no hydrocarbon production. This could be due to wells only producing water, or problems with reporting. 

In [78]:
well_prd_total = well_prod_df.groupby(['API_WELLNO']).sum() #sum the producing streams by api
well_prd_total['BOE'] = well_prd_total['BBLS_OIL_COND'] + well_prd_total['MCF_GAS']/5.8 #get total bbls of oil equivalent
well_prd_total = well_prd_total[well_prd_total['BOE']>0] #drop wells that didn't produce any oil and/or gas
api_list = list(well_prd_total.index) #get list of the api #'s that are associated with wells that had non-zero production
well_prod_df = well_prod_df[well_prod_df['API_WELLNO'].isin(api_list)]


In [79]:
lease_prd_total = lease_prod_df.groupby(['Lease_Unit']).sum() #sum the producing streams by lease unit
lease_prd_total['BOE'] = lease_prd_total['Oil_Prod'] + lease_prd_total['Gas_Prod']/5.8 #get total bbls of oil equivalent
lease_prd_total = lease_prd_total[lease_prd_total['BOE']>0] #drop leases that didn't produce any oil and/or gas
lease_list = list(lease_prd_total.index) #get list of the lease unit #'s that are associated with leases that had non-zero production
lease_prod_df = lease_prod_df[lease_prod_df['Lease_Unit'].isin(lease_list)]

# Horizontal Data

There are two ways that we can look at our data, we can choose to look through the lenses of leases which will contain a collection of 1 or more wells. The other is to perform analysis on a per well Basis. Each well will have a unique `API_WellNo` and each lease will be associated with a unique `Lease_Unit`. Our focus here will be on wells drilled horizontally, denoted by the `Slant` field in the `well_data_df`

In [80]:
hz_wells = well_data_df[well_data_df['Slant'] == 'H']
hz_wells_list = list(hz_wells['API_WellNo'])
hz_wells_prod = well_prod_df[well_prod_df['API_WELLNO'].isin(hz_wells_list)]
hz_wells_prod

,rpt_date,API_WELLNO,ST_FMTN_CD,Name_,Lease_Unit,OPNO,CoName,BBLS_OIL_COND,MCF_GAS,BBLS_WTR,DAYS_PROD,AMND_RPT,STATUS,dt_mod
545,01/31/1986,25021050590000,SO,Siluro-Ordovician,4541.0,NaN,NaN,21.0,0.0,57,5.0,False,P,NaN
4387,01/31/1986,25085214600000,RR,Red River,3257.0,NaN,NaN,610.0,660.0,83,28.0,False,P,NaN
4754,01/31/1986,25091215250000,RR,Red River,3276.0,NaN,NaN,1222.0,934.0,11023,31.0,False,P,NaN
7327,02/28/1986,25021050590000,SO,Siluro-Ordovician,4541.0,NaN,NaN,91.0,0.0,234,16.0,False,P,NaN
11122,02/28/1986,25085214600000,RR,Red River,3257.0,NaN,NaN,793.0,893.0,84,28.0,False,P,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028196,10/31/2021,25085217940000,TF,Three Forks,8445.0,794.0,"Foundation Energy Management, LLC",270.0,437.0,765,31.0,False,NaN,11/12/2021
5028197,10/31/2021,25085218010000,TF,Three Forks,8727.0,794.0,"Foundation Energy Management, LLC",0.0,0.0,0,0.0,False,NaN,11/12/2021
5028198,10/31/2021,25085218410000,TF,Three Forks,8766.0,794.0,"Foundation Energy Management, LLC",162.0,154.0,751,31.0,False,NaN,11/12/2021
5028199,10/31/2021,25085218550000,TF,Three Forks,8765.0,794.0,"Foundation Energy Management, LLC",130.0,399.0,795,31.0,False,NaN,11/12/2021


Finally, we can export the production and well header files for the horizontal wells for later use. 

In [81]:
hz_wells.to_csv('StateData\\hz_wells.csv', index = False)
hz_wells_prod.to_csv('StateData\\hz_wells_prod.csv', index = False)

# FracFocus Data
The next step is to pull in the data from FracFocus which will help give us a better idea of the materials and volumes that were used in stimulating these horizontal wells. FracFocus records start being widely reported around 2012. 

In [2]:
FracFocusRegistry_df_MT, registryupload_df_MT = pull_ff_data(state_name='Montana')

Now we can filter down to only records from Montana and get these dataframes into a useable state for analysis. The `StateNumber` for Montant is 25 which are the first two numbers in the API number. We will export for these files for later use.

In [ ]:
FracFocusRegistry_df_MT.head()
registryupload_df_MT.head()